In [119]:
import sys
!{sys.executable} -m pip install cssselect

In [4]:
!cp /Users/rikhoekstra/Downloads/republic---entities-_proef__project_2022-12-15_1348.zip .


In [279]:

import os
import zipfile

# this is the 'annotation 'subfolder of the unzipped export file, from where this notebook is run
anndir = './annotation/' #or whereever it is at

res = defaultdict(list)
tags = []
duplicates = 0
# the folder contains subfolders for all documents in inception 
# we loop over these subfolders
for docdir in os.listdir(anndir):
#     print(docdir,'\n')
    key = docdir.split('.')[0]
    try:
        fulldocdir = os.path.join(anndir,docdir)
    
# The document subfolder contain a zipfile per annotator, 
# The name of the zipfile is gibberish (like webanno10148686510395989892export.zip).
# First we loop over these zipfiles to collect all annotations on the document.

        for f in os.listdir(fulldocdir):
            file = os.path.join(fulldocdir,f)
            z = zipfile.ZipFile(file)
            l = z.namelist()# doc = etree.parse(z.open(l[1]))
            doc = etree.parse(z.open(l[1]))
            r = doc.getroot()
            nds = r.cssselect('[entity]')
            if len(nds)>0:
                txt = r.cssselect('[mimeType="text"]')[0].attrib['sofaString']
                for n in nds:
                    b=int(n.attrib['begin'])
                    e=int(n.attrib['end'])
                    content = txt[b:e]
                    record = {'begin':b,
                              'end':e,
                              'content':content,
                              'tag':n.attrib['entity'],
                              'prov':key}
                    res[key].extend(nds)
                    if record not in tags:
                        tags.append(record)
                    else:
                        duplicates += 1
    except (NotADirectoryError,BadZipfile): #for reasons of os interference mostly
        continue
#ts =  r.findall('.//')

In [282]:
duplicates

317

In [262]:
import pandas as pd

In [280]:
df = pd.DataFrame.from_records(tags)

In [281]:
df

,begin,end,content,tag,prov
0,46,106,Gecommitteerden van het Collegie ter Admiralit...,Organisatie,session-1727-02-07-ordinaris-num-1-resolution-11
1,102,106,Maze,Plaats - Politieke Entiteit,session-1727-02-07-ordinaris-num-1-resolution-11
2,262,267,Venlo,Plaats - Politieke Entiteit,session-1727-02-07-ordinaris-num-1-resolution-11
3,284,309,Collegie ter Admiraliteyt,Organisatie,session-1727-02-07-ordinaris-num-1-resolution-11
4,344,364,Magistraat van Venso,Persoonsaanduiding,session-1727-02-07-ordinaris-num-1-resolution-11
...,...,...,...,...,...
1199,66,77,Regensburgh,Plaats - Politieke Entiteit,session-1728-08-02-ordinaris-num-1-resolution-2
1200,82,121,ses en twintighsten der voorlede maandt,Datum,session-1728-08-02-ordinaris-num-1-resolution-2
1201,145,159,Griffier Fagel,Persoonsaanduiding,session-1728-08-02-ordinaris-num-1-resolution-2
1202,145,153,Griffier,Hoedanigheid,session-1728-08-02-ordinaris-num-1-resolution-2


In [267]:
df.sort_values('tag')

,begin,end,content,tag,prov
143,30,120,"Heeren van Welderen, ende andere haer Hoogh Mo...",Commissie,session-1705-12-24-ordinaris-num-1-resolution-11
1141,484,571,"Heeren Torek, en andere haar Hoogh Mogende Ged...",Commissie,session-1729-06-30-ordinaris-num-1-resolution-14
276,30,116,"Heeren de FEaet, ende andere haer Hoogh Mogend...",Commissie,session-1706-10-18-ordinaris-num-1-resolution-10
503,735,830,"Heeren van Wynbergen , ende andere haar Hoogh ...",Commissie,session-1722-08-04-ordinaris-num-1-resolution-10
55,516,606,"Heeren van Oldersom, ende andere haer Hoogh Mo...",Commissie,session-1712-10-20-ordinaris-num-1-resolution-33
...,...,...,...,...,...
973,566,631,haer Hoogh. Mogende Resolutie den 15. deser di...,Resolutieverwijzing,session-1705-07-17-ordinaris-num-1-resolution-18
572,4011,4224,haar Hoogh Mogende Resolutien van den een en t...,Resolutieverwijzing,session-1731-07-17-ordinaris-num-1-resolution-5
294,176,223,Resolutie commissoriael van den twaelfden deser,Resolutieverwijzing,session-1706-10-18-ordinaris-num-1-resolution-10
421,161,224,Resolutie commissoriael van den dertighsten de...,Resolutieverwijzing,session-1712-02-06-ordinaris-num-1-resolution-12


In [283]:
df.sort_values('tag').to_csv('/Users/rikhoekstra/Downloads/tags20221216.tsv', sep='\t')

In [287]:
df.sort_values('tag').to_excel('/Users/rikhoekstra/Downloads/tags20221216.xlsx')

In [284]:
df.content.value_counts().to_frame().head(10)

,content
Commisen Generaal,28
Zeelandt,27
Vrieslandt,21
Collegien ter Admiraliteyt,17
Fiscaal van de Generaliteyt,13
Raadt van State,9
Resident,9
Amsterdam,9
Griffier,8
Ambassadeur,8


In [285]:
df.tag.value_counts()


Hoedanigheid                   335
Plaats - Politieke Entiteit    318
Persoonsaanduiding             194
Organisatie                    167
Datum                          108
Resolutieverwijzing             29
Overige Naam                    26
Commissie                       18
Locatie                          9
Name: tag, dtype: int64